In [ ]:
import numpy as np
import pandas as pd
import cv2
import cv2 as cv
import os
import matplotlib.pyplot as plt
import seaborn as sns 

In [ ]:
XData = []
yData = []

age_list = dict()

for dirname, _, filenames in os.walk('dataset'):
    for i,filename in enumerate(filenames):
        content = filename.split("_")
        age = content[0]
        
        try:
        
            if(age_list[age] < 110):
                yData.append(content[0])

                img = cv2.imread(os.path.join(dirname, filename))
                img = img / 25
                img = cv2.resize(img, (150, 150))
                XData.append(img)

                age_list[age] = age_list[age] + 1
            
        
        except:
            age_list[age] = 0
        
        if(i % 100 == 0):
            print(i, end = " ")

In [ ]:
# perform EDA on the collected data
# Descriptive Statistics (for age)
age_values = [int(age) for age in yData]
age_series = pd.Series(age_values)
age_summary = age_series.describe()

# Data Visualization
# Plot age distribution
plt.figure(figsize=(10, 6))
sns.histplot(age_values, kde=True)
plt.title('Age Distribution')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

In [ ]:
age_summary

In [ ]:
age_series.head()

In [ ]:
age_series.shape

In [ ]:
age_series.info()

In [ ]:
age_series.nunique()

**Histogram Equalization**

In [ ]:
# Function to calculate normalized histogram and CDF for a given age value
def calculate_histogram_and_cdf(age_value):
    img_sum = np.array([0] * 40000)
    count = 0
    for dirname, _, filenames in os.walk('dataset'):
        for i, filename in enumerate(filenames):
            content = filename.split("_")
            age = content[0]
            if (age == age_value):
                img = np.array(cv2.imread(os.path.join(dirname, filename), cv2.IMREAD_GRAYSCALE))
                img_sum = np.add(img_sum, img.flatten())
                count = count + 1
    average = np.divide(img_sum, count)

    # Calculate histogram of the 'average' array
    hist, bins = np.histogram(average, 256, [0, 256])

    # Normalize the histogram's y-axis values to the range [0, 1]
    normalized_hist = hist / np.max(hist)

    # Calculate cumulative distribution function (CDF)
    cdf = hist.cumsum()
    cdf_normalized = cdf / cdf.max()

    return bins, normalized_hist, cdf_normalized

# Calculate histogram and CDF for age 1 and age 100
bins_age_1, normalized_hist_age_1, cdf_normalized_age_1 = calculate_histogram_and_cdf('1')
bins_age_100, normalized_hist_age_100, cdf_normalized_age_100 = calculate_histogram_and_cdf('100')

# Plot both histograms and CDFs on the same graph for comparison
plt.figure(figsize=(10, 6))
plt.plot(cdf_normalized_age_1, color='b', label='Age 1 CDF')
plt.plot(cdf_normalized_age_100, color='r', label='Age 100 CDF')
plt.bar(bins_age_1[:-1], normalized_hist_age_1, width=(bins_age_1[1] - bins_age_1[0]), color='b', alpha=0.5, label='Age 1 Histogram')
plt.bar(bins_age_100[:-1], normalized_hist_age_100, width=(bins_age_100[1] - bins_age_100[0]), color='r', alpha=0.5, label='Age 100 Histogram')
plt.xlim([0, 256])
plt.ylim([0, 1])
plt.legend()
plt.xlabel('Pixel Intensity')
plt.ylabel('Normalized Frequency / CDF')
plt.title('Comparison of Normalized Histograms and CDFs for Age 1 and Age 100')
plt.show()

In [ ]:
# Function to calculate average brightness for a given age value
def calculate_average_brightness(age_value):
    brightness_values = []
    for dirname, _, filenames in os.walk('dataset'):
        for filename in filenames:
            content = filename.split("_")
            age = int(content[0])
            if age == age_value:
                img = cv2.imread(os.path.join(dirname, filename), cv2.IMREAD_GRAYSCALE)
                # Calculate average pixel intensity (brightness)
                avg_brightness = np.mean(img)
                brightness_values.append(avg_brightness)
    return brightness_values

# Calculate average brightness for age 10 and age 60
brightness_age_10 = calculate_average_brightness(10)
brightness_age_60 = calculate_average_brightness(60)

# Plotting the brightness distribution
plt.figure(figsize=(8, 6))
plt.hist(brightness_age_10, bins=50, alpha=0.5, color='blue', label='Age 10')
plt.hist(brightness_age_60, bins=50, alpha=0.5, color='red', label='Age 60')
plt.xlabel('Average Pixel Intensity (Brightness)')
plt.ylabel('Frequency')
plt.title('Brightness Distribution for Age 10 and Age 60')
plt.legend()
plt.show()

In [ ]:
# Function to perform Canny edge detection for a given age value
def perform_edge_detection(age_value):
    edge_counts = []
    for dirname, _, filenames in os.walk('dataset'):
        for filename in filenames:
            content = filename.split("_")
            age = int(content[0])
            if age == age_value:
                img = cv2.imread(os.path.join(dirname, filename), cv2.IMREAD_GRAYSCALE)
                # Apply Canny edge detection
                edges = cv2.Canny(img, 100, 200)
                # Count the number of edge pixels
                edge_counts.append(np.sum(edges) / 255)  # Dividing by 255 to count white pixels (edges)
    return edge_counts

# Perform edge detection for age 10 and age 60
edges_age_10 = perform_edge_detection(10)
edges_age_60 = perform_edge_detection(60)

# Plotting the edge distribution
plt.figure(figsize=(8, 6))
plt.hist(edges_age_10, bins=50, alpha=0.5, color='blue', label='Age 10')
plt.hist(edges_age_60, bins=50, alpha=0.5, color='red', label='Age 60')
plt.xlabel('Number of Edge Pixels')
plt.ylabel('Frequency')
plt.title('Edge Distribution for Age 10 and Age 60 (Canny Edge Detection)')
plt.legend()
plt.show()